In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install opencv-python-headless
!pip install scikit-learn
!pip install efficientnet
!pip install innvestigate
!pip install pandas

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
import cv2

from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score, confusion_matrix, ConfusionMatrixDisplay

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.backend
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_eff
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D

tf.compat.v1.disable_eager_execution()

import innvestigate
import innvestigate.utils as iutils

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/face_forensic'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_010_005.mp4_frame1.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_021_312.mp4_frame1.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_021_312.mp4_frame2.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_005_010.mp4_frame3.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_012_026.mp4_frame1.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_040_997.mp4_frame1.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_039_058.mp4_frame3.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_028_068.mp4_frame2.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_041_063.mp4_frame5.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_039_058.mp4_frame2.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_044_945.mp4_frame2.jpg
/content/drive/MyDrive/face_forensic/Face2Face/FAKE-Face2face_049

In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter("ignore")

%matplotlib inline

In [4]:
faceforensis_real_path = "/content/drive/MyDrive/face_forensic/REAL"
faceforensis_fake_deepfake = "/content/drive/MyDrive/face_forensic/Deepfake"
faceforensis_fake_deepfakedetection = "/content/drive/MyDrive/face_forensic/DeepfakeDetection"
faceforensis_fake_face2face = "/content/drive/MyDrive/face_forensic/Face2Face"
faceforensis_fake_faceshifter = "/content/drive/MyDrive/face_forensic/FaceShifter"
faceforensis_fake_faceswap = "/content/drive/MyDrive/face_forensic/FaceSwap"
faceforensis_fake_neuraltextures = "/content/drive/MyDrive/face_forensic/NeuralTextures"

# models
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
eff_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

58889256/58889256 [==============================] - 0s 0us/step


Instructions for updating:
Colocations handled automatically by placer.


16705208/16705208 [==============================] - 0s 0us/step


In [5]:
import os

def listar_arquivos(pasta):
    if not os.path.exists(pasta):
        print(f"[ERRO] Pasta não encontrada: {pasta}")
        return

    arquivos = os.listdir(pasta)
    print(f"Total de arquivos em {pasta}: {len(arquivos)}")
    print("Exemplo de arquivos:", arquivos[:5])

listar_arquivos(faceforensis_real_path)
listar_arquivos(faceforensis_fake_deepfake)


Total de arquivos em /content/drive/MyDrive/face_forensic/REAL: 250
Exemplo de arquivos: ['REAL_020.mp4_frame3.jpg', 'REAL_009.mp4_frame1.jpg', 'REAL_017.mp4_frame4.jpg', 'REAL_018.mp4_frame1.jpg', 'REAL_002.mp4_frame2.jpg']
Total de arquivos em /content/drive/MyDrive/face_forensic/Deepfake: 250
Exemplo de arquivos: ['FAKE-Deepfake_022_489.mp4_frame1.jpg', 'FAKE-Deepfake_037_072.mp4_frame1.jpg', 'FAKE-Deepfake_036_035.mp4_frame3.jpg', 'FAKE-Deepfake_001_870.mp4_frame2.jpg', 'FAKE-Deepfake_003_000.mp4_frame1.jpg']


In [6]:
import innvestigate.utils.visualizations as ivis

channels_first = keras.backend.image_data_format() == "channels_first"
color_conversion = "BGRtoRGB"

def postprocess(X, color_conversion, channels_first):
  X = X.copy()
  X = iutils.postprocess_images(
      X, color_coding=color_conversion, channels_first=channels_first
  )
  return X

def heatmap(X):
  return ivis.heatmap(X)

In [7]:
def plot_confusion_matrix(y_test, y_pred):
  cm = confusion_matrix(y_test, y_pred)
  disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["REAL", "FAKE"])
  plt.figure(figsize=(5, 5))
  disp.plot(cmap=plt.cm.Blues)
  plt.title("Confusion Matrix")
  plt.show()

In [8]:
def extract_features_from_image(image_path):
  if not os.path.isfile(image_path):
    print(f"[ERRO] O arquivo {image_path} não é válido ou não é uma imagem.")
    return None

  img = cv2.imread(image_path)
  if img is None:
    print(f"[ERRO] Não foi possível carregar a imagem {image_path}.")
    return None

  img_resized = cv2.resize(img, (224, 224))
  img_rgb = cv2.cvtColor(img_resized, cv2.COLOR_BGR2RGB)

  img_batch_vgg = preprocess_vgg(np.expand_dims(img_rgb.astype('float32'), axis = 0))
  img_batch_resnet = preprocess_resnet(np.expand_dims(img_rgb.astype('float32'), axis=0))
  img_batch_eff = preprocess_eff(np.expand_dims(img_rgb.astype('float32'), axis=0))

  vgg_feat = vgg_model.predict(img_batch_vgg, verbose=0).flatten()
  res_feat = resnet_model.predict(img_batch_resnet, verbose=0).flatten()
  eff_feat = eff_model.predict(img_batch_eff, verbose=0).flatten()

  combined = np.concatenate([vgg_feat, res_feat, eff_feat])

  return combined

def generate_model(real_path, fake_path):
  X_train, y_train = [], []
  X_test, y_test = [], []

  for label, path in [("REAL", real_path), ("FAKE", fake_path)]:
    image_files = os.listdir(path)
    test_files = image_files[:50]
    train_files = image_files[50:]

    for image_name in tqdm(test_files, desc=f"{label} (TESTE)"):
      image_path = os.path.join(path, image_name)
      feat = extract_features_from_image(image_path)
      if feat is not None:
        X_test.append(feat)
        y_test.append(0 if label == "REAL" else 1)
      else:
        print(f"[INFO] Não foram extraídas características de {image_path}")

    for image_name in tqdm(train_files, desc=f"{label} (TREINO)"):
      image_path = os.path.join(path, image_name)
      feat = extract_features_from_image(image_path)
      if feat is not None:
        X_train.append(feat)
        y_train.append(0 if label == "REAL" else 1)
      else:
        print(f"[INFO] Naõ foram extraídas características de {image_path}")

  if len(X_train) == 0 or len(X_test) == 0:
    print("Erro: Não foram extraídas características suficientes para o treino ou teste.")
    return None, None

  X_train = np.array(X_train)
  y_train = np.array(y_train)
  X_test = np.array(X_test)
  y_test = np.array(y_test)

  # PCA + SVM
  pca = PCA(n_components=1)
  X_train_pca = pca.fit_transform(X_train)
  X_test_pca = pca.transform(X_test)

  svm = SVC(kernel='linear')
  svm.fit(X_train_pca, y_train)

  y_pred = svm.predict(X_test_pca)

  print("Accuracy:", accuracy_score(y_test, y_pred))
  print("F1-score (macro):", f1_score(y_test, y_pred, average='macro'))

  if hasattr(svm, "decision_function"):
    y_scores = svm.decision_function(X_test_pca)
    auc = roc_auc_score(y_test, y_scores)
    print("AUC:", auc)
  else:
    print("O modelo SVM não possui decision_function para calcular a AUC.")

  print("Classification Report:\n", classification_report(y_test, y_pred))

  return svm, pca



In [ ]:
# Chamada da função
print("Deepfake")
svm_deepfake, pca_deepfake = generate_model(faceforensis_real_path, faceforensis_fake_deepfake)
print("DeepfakeDetection")
svm_deepfakedetection, pca_deepfakedetection = generate_model(faceforensis_real_path, faceforensis_fake_deepfakedetection)
print("Face2Face")
svm_face2face, pca_face2face = generate_model(faceforensis_real_path, faceforensis_fake_face2face)
print("FaceShifter")
svm_faceshifter, pca_faceshifter = generate_model(faceforensis_real_path, faceforensis_fake_faceshifter)
print("FaceSwap")
svm_faceswap, pca_faceswap = generate_model(faceforensis_real_path, faceforensis_fake_faceswap)
print("NeuralTextures")
svm_neuratextures, pca_neuratextures = generate_model(faceforensis_real_path, faceforensis_fake_neuraltextures)

Deepfake


FAKE (TREINO):  52%|█████▏    | 104/200 [01:37<01:33,  1.03it/s]